In [1]:
import pandas as pd
import os
import random
import altair as alt

In [2]:
data_dir = '.'
filename = 'US_mobility_long.zip'

mob = pd.read_csv(
    os.path.join(data_dir, filename),
    compression='zip',
    parse_dates=['date']
)

In [3]:
# create list of non-null unique state names
states = list(
    mob.loc[
        mob.sub_region_1.notna(),
        'sub_region_1'
    ].unique()
)

In [4]:
def draw_plot():
    # sample 5 states
    sample_states = random.sample(states, 5)
    
    # filter
    plot_data = mob.loc[
        (mob.sub_region_1.isin(sample_states)) &
        (mob.sub_region_2.isna()) & # return statewide instead of county-level
        (mob.date > mob.date.max() - pd.Timedelta(100, unit='D')) & # 100 days back from current date
        (mob.activity != 'parks') # hurts scaling of facets
    ].reset_index()
    
    # draw facets
    alt.Chart(plot_data).mark_area(opacity=0.5).encode(
        x='date:T',
        y='percent_change:Q',
        color='activity:N'
    ).properties(
        width=130,
        height=130
    ).facet(
        column='activity:N',
        row='sub_region_1:N'
    ).configure_axis(
        grid=False
    ).configure_view(
        strokeWidth=0
    ).display()

In [5]:
draw_plot()

alt.FacetChart(...)